# Introduction: 

Electrical systems vary significantly around the world, including differences in plug types, voltages, and frequencies. While efforts have been made to standardize these systems in certain regions, many variations still exist globally. This notebook aims to scrape data from the [Wikipedia Mains electricity by country](https://en.wikipedia.org/wiki/Mains_electricity_by_country) page , providing a concise summary of the key electrical characteristics used in different countries.


## Setup and Libraries:

In this section, we will import the required libraries, including BeautifulSoup for parsing HTML, pandas for data manipulation and storage, re for regular expressions, and requests for making HTTP requests.

This version gives a more detailed explanation of why each library is being used, which can be helpful for readers to understand their purposes

In [1]:
# This library is used to send HTTP requests to websites, allowing us to fetch their HTML content.
import requests  

# BeautifulSoup helps to parse the HTML content and extract specific elements like tables, headings, and text.
from bs4 import BeautifulSoup  

# Pandas is used to store and manipulate data in a tabular format (DataFrame), making it easy to clean and export data.
import pandas as pd  

# The re library provides support for regular expressions, which are used to search, match, and manipulate text patterns.
import re  

## Requesting the Webpage:

In this section, we define a variable that holds the URL of the webpage we want to scrape. Using the requests library, we send an HTTP GET request to this URL. A successful response will be indicated by a status code of <Response [200]>, which means the page was retrieved successfully and is ready for further processing.

In [2]:
URL = "https://en.wikipedia.org/wiki/Mains_electricity_by_country"

response = requests.get(URL)
response

<Response [200]>

## Parsing the HTML Content:

Since we successfully retrieved the webpage, we can proceed with parsing the HTML content. Using BeautifulSoup, we transform the raw HTML into a structured format that allows us to easily navigate and extract specific elements, such as tables, headings, or text

In [3]:
soup = BeautifulSoup(response.content, "html.parser")

## Finding the Table :

Using the browser’s Developer Tools, we inspect the structure of the webpage to understand how the content is organized. This helps us locate the table containing the information we need. Once identified, we use BeautifulSoup to extract the table and further break down its components, such as headers and rows, to ensure that all relevant data is captured.

We filter out elements with **colspan="2"** to handle the multi-level headers in the table, allowing us to focus on the specific headers and data rows needed for our analysis.

In [4]:
# Find table
table = soup.find('table')  
# Find all <th> elements
th_elements = soup.find_all('th')

# Filter out <th> elements that have colspan="2" and extract text
th_texts = [
    th.get_text(strip=True) 
    for th in th_elements 
    if th.get('colspan') != '2'
]

# Print the list of extracted text
print(th_texts)

['Country or territory', 'Plug type[a]', 'National plugstandard[2]', 'Fre\xadquen\xadcy[7]', 'Notes', 'Resi\xaddential[7]', 'Three-phase[8]', 'vteElectricity delivery', 'Concepts', 'Sources', 'Non-renewable', 'Renewable', 'Generation', 'Transmissionanddistribution', 'Failure modes', 'Protectivedevices', 'Economicsand policies', 'Statistics andproduction']


### Verifing and Cleaing Headers: 

Upon inspection, we find additional elements among the headers that are not relevant or useful for our analysis. To ensure that our data remains clean and focused, we remove these unnecessary elements, retaining only the headers that contain valuable information.

In [5]:
# Extract the first 7 elements from the th_texts list
first_seven_th_texts = th_texts[:7]

# Print the first 8 elements
print(first_seven_th_texts)

['Country or territory', 'Plug type[a]', 'National plugstandard[2]', 'Fre\xadquen\xadcy[7]', 'Notes', 'Resi\xaddential[7]', 'Three-phase[8]']


The headers contain unwanted characters such as newline characters (\n), soft hyphens (\xad), and additional links. It is necessary to remove these elements to ensure the headers are clean and properly formatted for further analysis.

In [6]:
# Function to clean headers
def clean_headers(headers):
    cleaned_headers = []
    for header in headers:
        # Remove all text inside square brackets [] and any newlines or soft hyphens (\xad)
        clean_header = re.sub(r'\[.*?\]|\n|\xad', '', header)
        cleaned_headers.append(clean_header.strip())
    return cleaned_headers

# Clean the headers
cleaned_heads = clean_headers(first_seven_th_texts)
print(cleaned_heads)

['Country or territory', 'Plug type', 'National plugstandard', 'Frequency', 'Notes', 'Residential', 'Three-phase']


### Manually Reordering and Naming the Headers:

Since the headers do not appear in the same order as shown on the website, we manually rearrange them to match the desired structure. This ensures consistency between the scraped data and the original table format on the webpage, making our data more accurate and easy to analyze.

In [7]:
cleaned_heads = ['Country or territory', 'Plug type', 'National plugstandard','Residential Voltage (V)', 'Three-phase Voltage (V)', 'Frequency (Hz)', 'Notes']

### Extracting Data from the Table:

While extracting the remaining elements of the table, we noticed that the first few rows contain repeated headers instead of data. To ensure the data is clean and relevant, we remove these redundant header rows, keeping only the rows that contain the actual data we need for analysis.


In [8]:
# Find the <tbody> element in the table
tbody = soup.find('tbody')
# Find the <re> element in the table, which are the rows of the table
rows = tbody.find_all('tr')

# Create table
table_data = []
# Insert text to the table
for tr in rows:
        # Extract all cells (<td> and <th>) inside each row
    cells = [cell.get_text(strip=True) for cell in tr.find_all(['td', 'th'])]
    table_data.append(cells)
# Remove the first two rows
table_data = table_data[2:]     

In [9]:
# Create the DataFrame
Df_table_data = pd.DataFrame(table_data, columns=cleaned_heads)
Df_table_data

,Country or territory,Plug type,National plugstandard,Residential Voltage (V),Three-phase Voltage (V),Frequency (Hz),Notes
0,Afghanistan,"C, F",,220V,380 V,50Hz,
1,Albania,"C, F",,230 V,400 V,50 Hz,
2,Algeria,"C, F",,240 V,400 V,50 Hz,
3,American Samoa,"A, B, F, I",,120 V,208 V,60 Hz,
4,Andorra,"C, F",,230 V,400 V,50 Hz,
...,...,...,...,...,...,...,...
223,Venezuela,"A, B",,120 V208 V240 V,115/220 V220/440 V230/460 V[72],60 Hz,
224,Vietnam,"A, B, C, D, F, G",TCVN 6188-1,220 V,380 V,50 Hz,Majority of households use unearthed hybrid so...
225,Yemen,"A, D, G",,240 V,400 V,50 Hz,
226,Zambia,"C, D, G",,230 V,400 V,50 Hz,


### Removing Links from Data:

Since some of the data contains embedded links, we need to remove these to ensure the dataset is clean and focused on the textual information. 

In [10]:
# Create function to remove brackets and value inside using Regex
def remove_specific_bracket_value(value):
    return re.sub(r'\[.*?\]', '', value).strip()

# Apply the function only to specific columns
Df_table_data['Plug type'] = Df_table_data['Plug type'].apply(remove_specific_bracket_value)
Df_table_data['National plugstandard'] = Df_table_data['National plugstandard'].apply(remove_specific_bracket_value)
Df_table_data['Residential Voltage (V)'] = Df_table_data['Residential Voltage (V)'].apply(remove_specific_bracket_value)
Df_table_data['Three-phase Voltage (V)'] = Df_table_data['Three-phase Voltage (V)'].apply(remove_specific_bracket_value)
Df_table_data['Frequency (Hz)'] = Df_table_data['Frequency (Hz)'].apply(remove_specific_bracket_value)
Df_table_data


,Country or territory,Plug type,National plugstandard,Residential Voltage (V),Three-phase Voltage (V),Frequency (Hz),Notes
0,Afghanistan,"C, F",,220V,380 V,50Hz,
1,Albania,"C, F",,230 V,400 V,50 Hz,
2,Algeria,"C, F",,240 V,400 V,50 Hz,
3,American Samoa,"A, B, F, I",,120 V,208 V,60 Hz,
4,Andorra,"C, F",,230 V,400 V,50 Hz,
...,...,...,...,...,...,...,...
223,Venezuela,"A, B",,120 V208 V240 V,115/220 V220/440 V230/460 V,60 Hz,
224,Vietnam,"A, B, C, D, F, G",TCVN 6188-1,220 V,380 V,50 Hz,Majority of households use unearthed hybrid so...
225,Yemen,"A, D, G",,240 V,400 V,50 Hz,
226,Zambia,"C, D, G",,230 V,400 V,50 Hz,


### Saving the Cleaned Dataset:

Now that we have cleaned the dataset, we save it as a CSV file. This allows us to easily download and share the data or use it for further analysis.

In [11]:
# Save the DataFrame as a CSV file 
csv_name = '/kaggle/working/mains_electricity_by_country.csv'
Df_table_data.to_csv(csv_name, index=False)